# BNL Development Notebook

This notebook is for developing and testing the BNL package.


In [ ]:
%load_ext autoreload
%autoreload 2

import bnl
from random import choice

slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")
track = slm_ds[11]

available_refs = list(track.refs)
available_ests = list(track.ests)
print(available_refs, available_ests)

est = track.ests[choice(available_ests)]
ref = track.refs[choice(available_refs)]

In [ ]:
est = track.ests[choice(available_ests)]
est1 = est.prune_layers(relabel=False)
est2 = est.prune_layers(relabel=True)
est.plot().show()
est1.plot().show()
est2.plot().show()

In [ ]:
mock_empty_layer = bnl.S(
    bs=[ref.start, ref.end], labels=[None], 
    name='empty'
)
mock_layer = bnl.S(
    bs=ref[1].bs, labels=[f'lab_{lab}' for lab in ref[1].labels], 
    name='redundant'
)
mock_ref = bnl.MS(
    layers=[mock_empty_layer] + ref.layers + [mock_layer], 
    name='mock_ref'
)

mock_ref.plot().show()
mock_ref.prune_layers().plot().show()

In [ ]:
# First figure: we need to show for T-measure, what's it's deal with monotonicity.
import mir_eval
import numpy as np
from random import choice
from bnl.data import Dataset
from plotly.subplots import make_subplots

slm_ds = Dataset()
track = slm_ds[18]

est = track.ests[choice(list(track.ests))]
ref = track.refs[choice(list(track.refs))]
aligned_est = est.align(span=ref)


In [ ]:
mono_est_count = aligned_est.contour('count').level().to_ms()
mono_est_depth = aligned_est.contour('depth').level().to_ms()
mono_est_prob = aligned_est.contour('prob').level().to_ms()


In [ ]:
fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.04,
    subplot_titles=["Reference", "Estimated Raw", "Count", "Depth"],
    row_heights=[0.1, 0.3, 0.2, 0.2],
)
[fig.add_trace(labeld_timespans, row=1, col=1) for labeld_timespans in ref.plot().data]
[fig.add_trace(labeld_timespans, row=2, col=1) for labeld_timespans in aligned_est.plot().data]
[fig.add_trace(labeld_timespans, row=3, col=1) for labeld_timespans in mono_est_count.plot().data]
[fig.add_trace(labeld_timespans, row=4, col=1) for labeld_timespans in mono_est_depth.plot().data]
fig.update_layout(barmode="overlay", legend_visible=False, height=1000, width=800)
fig.update_xaxes(autorangeoptions=dict(minallowed=ref.start.time, maxallowed=ref.end.time))
fig.show()

In [ ]:
import mir_eval
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

mono_est_depth = aligned_est.to_contour('depth').to_multisegment()

a = mir_eval.hierarchy.tmeasure(ref.itvls, aligned_est.itvls[2:6], transitive=True)
b = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_depth.itvls[2:6], transitive=True)
c = mir_eval.hierarchy.tmeasure(ref.itvls, aligned_est.itvls[2:6], transitive=False)
d = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_depth.itvls[2:6], transitive=False)


print(np.array([a, b, c, d]))

In [ ]:
from mir_eval.hierarchy import tmeasure

annos = [
    ref.scrub_labels(), 
    MS(aligned_est[2:6], 'Raw').scrub_labels(), 
    MS(mono_est_depth[2:6], 'Mono')
]

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    subplot_titles=["Reference", "Raw", "Mono"]
)
fig.update_layout(
    barmode="overlay", legend_visible=False, height=650, width=650, 
    margin=dict(l=10, r=10, t=50, b=10)
)
fig.update_xaxes(
    autorangeoptions=dict(minallowed=ref.start.time, maxallowed=ref.end.time)
)

for row, anno in enumerate(annos, start=1):
    if row != 1:
        print(f"{anno}: reduced (tran=False)\n  {tmeasure(ref.itvls, anno.itvls)}")
        print(f"{anno}: full (tran=True)\n  {tmeasure(ref.itvls, anno.itvls, transitive=True)}")
    for bar_trace in anno.plot().data:
        fig.add_trace(bar_trace, row=row, col=1)

fig.show()

In [ ]:
# Study the effect of monotonic casting and T-measure